In [1]:
from pathlib import Path
import GEOparse
import glob
import json
import os
import statistics
import pandas as pd
import numpy as np
import xgboost as xgb
import random
from collections import Counter

In [5]:
dir_path_rel = Path(os.path.join('.', 'gds-soft'))
dir_path_abs = dir_path_rel.absolute()
file_path = os.path.join(str(dir_path_abs) + '/GDS*.soft.gz')
gds_files = glob.glob(file_path)

len(gds_files)

1772

So, upto this we can see that all the GDS's names are read by the program
#### Now at first, we go on to test the merging  of only two of the GDS files:

In [ ]:
# gds_dims = {}
# for gds_file in gds_files[0:2]:
#     print(gds_file)
#     gse = GEOparse.get_GEO(filepath=gds_file)
#     gds_dims[gds_file.split('/')[-1]] =  gse.table.shape

# for key, value in gds_dims.items():
#         print(key + '     ' + str(value))

In [2]:
gds4380 = GEOparse.get_GEO(filepath="D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS4380.soft.gz")
GDS2 = gds4380.table
GDS2.shape, GDS2.columns[:3]

10-Apr-2019 12:35:12 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS4380.soft.gz: 
10-Apr-2019 12:35:12 DEBUG GEOparse - DATABASE: Geo
10-Apr-2019 12:35:12 DEBUG GEOparse - DATASET: GDS4380
10-Apr-2019 12:35:12 DEBUG GEOparse - SUBSET: GDS4380_1
10-Apr-2019 12:35:12 DEBUG GEOparse - SUBSET: GDS4380_2
10-Apr-2019 12:35:12 DEBUG GEOparse - DATASET: GDS4380


((54675, 37), Index(['ID_REF', 'IDENTIFIER', 'GSM757714'], dtype='object'))

In [3]:
GDS2.to_csv("../gds4380.csv", index=False)

In [10]:
gse12 = GEOparse.get_GEO(filepath="D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS12.soft.gz")
GDS1 = gse12.table
GDS1.shape, GDS1.columns[:3]

08-Apr-2019 21:11:49 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS12.soft.gz: 
08-Apr-2019 21:11:49 DEBUG GEOparse - DATABASE: Geo
08-Apr-2019 21:11:49 DEBUG GEOparse - DATASET: GDS12
08-Apr-2019 21:11:49 DEBUG GEOparse - SUBSET: GDS12_1
08-Apr-2019 21:11:49 DEBUG GEOparse - SUBSET: GDS12_2
08-Apr-2019 21:11:49 DEBUG GEOparse - DATASET: GDS12


((17588, 10), Index(['ID_REF', 'IDENTIFIER', 'GSM834'], dtype='object'))

In [36]:
ex_large_dataset = GDS1
ex_large_dataset = pd.merge(ex_large_dataset, GDS2, on='IDENTIFIER', how='outer')
ex_large_dataset.shape

(69676, 46)

In [38]:
#ex_large_dataset = ex_large_dataset.drop(columns=['ID_REF_x','ID_REF_y'])
ex_large_dataset.columns

Index(['ID_REF_x', 'IDENTIFIER', 'GSM834', 'GSM835', 'GSM836', 'GSM837',
       'GSM838', 'GSM839', 'GSM840', 'GSM841', 'ID_REF_y', 'GSM757714',
       'GSM757721', 'GSM757722', 'GSM757723', 'GSM757730', 'GSM757733',
       'GSM757735', 'GSM757740', 'GSM757741', 'GSM757746', 'GSM757713',
       'GSM757715', 'GSM757716', 'GSM757717', 'GSM757718', 'GSM757719',
       'GSM757720', 'GSM757724', 'GSM757725', 'GSM757726', 'GSM757727',
       'GSM757728', 'GSM757729', 'GSM757731', 'GSM757732', 'GSM757734',
       'GSM757736', 'GSM757737', 'GSM757738', 'GSM757739', 'GSM757742',
       'GSM757743', 'GSM757744', 'GSM757745', 'GSM757747'],
      dtype='object')

### We saw that merging is done right:
### In following cell we see how mane NaN values exist in each column

In [30]:
nan_values = ex_large_dataset.shape[0] - 1 - ex_large_dataset.iloc[:,1:].count()
nan_values

GSM834       40818
GSM835       39589
GSM836       39376
GSM837       40014
GSM838       38568
GSM839       41596
GSM840       41171
GSM841       39273
GSM757714     9605
GSM757721     9605
GSM757722     9605
GSM757723     9605
GSM757730     9605
GSM757733     9605
GSM757735     9605
GSM757740     9605
GSM757741     9605
GSM757746     9605
GSM757713     9605
GSM757715     9605
GSM757716     9605
GSM757717     9605
GSM757718     9605
GSM757719     9605
GSM757720     9605
GSM757724     9605
GSM757725     9605
GSM757726     9605
GSM757727     9605
GSM757728     9605
GSM757729     9605
GSM757731     9605
GSM757732     9605
GSM757734     9605
GSM757736     9605
GSM757737     9605
GSM757738     9605
GSM757739     9605
GSM757742     9605
GSM757743     9605
GSM757744     9605
GSM757745     9605
GSM757747     9605
dtype: int64

### We try on merging 2 more GDS files with larger dataset to test

In [40]:
gse77 = GEOparse.get_GEO(filepath="D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS77.soft.gz")
GDS3 = gse77.table
GDS3.shape, GDS3.columns

08-Apr-2019 21:46:35 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS77.soft.gz: 
08-Apr-2019 21:46:35 DEBUG GEOparse - DATABASE: Geo
08-Apr-2019 21:46:35 DEBUG GEOparse - DATASET: GDS77
08-Apr-2019 21:46:35 DEBUG GEOparse - SUBSET: GDS77_1
08-Apr-2019 21:46:35 DEBUG GEOparse - SUBSET: GDS77_2
08-Apr-2019 21:46:35 DEBUG GEOparse - DATASET: GDS77


((24192, 4),
 Index(['ID_REF', 'IDENTIFIER', 'GSM1802', 'GSM1803'], dtype='object'))

In [39]:
print ("previous shape: %d,%d"%(ex_large_dataset.shape[0],ex_large_dataset.shape[1]))
ex_large_dataset = pd.merge(ex_large_dataset, GDS3, on='IDENTIFIER', how='outer')
print ("new shape: %d,%d"%(ex_large_dataset.shape[0],ex_large_dataset.shape[1]))
ex_large_dataset.columns

previous shape: 69676,46
new shape: 93515,49


Index(['ID_REF_x', 'IDENTIFIER', 'GSM834', 'GSM835', 'GSM836', 'GSM837',
       'GSM838', 'GSM839', 'GSM840', 'GSM841', 'ID_REF_y', 'GSM757714',
       'GSM757721', 'GSM757722', 'GSM757723', 'GSM757730', 'GSM757733',
       'GSM757735', 'GSM757740', 'GSM757741', 'GSM757746', 'GSM757713',
       'GSM757715', 'GSM757716', 'GSM757717', 'GSM757718', 'GSM757719',
       'GSM757720', 'GSM757724', 'GSM757725', 'GSM757726', 'GSM757727',
       'GSM757728', 'GSM757729', 'GSM757731', 'GSM757732', 'GSM757734',
       'GSM757736', 'GSM757737', 'GSM757738', 'GSM757739', 'GSM757742',
       'GSM757743', 'GSM757744', 'GSM757745', 'GSM757747', 'ID_REF', 'GSM1802',
       'GSM1803'],
      dtype='object')

In [41]:
gse78 = GEOparse.get_GEO(filepath="D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS78.soft.gz")
GDS4 = gse78.table
GDS4.shape, GDS4.columns

08-Apr-2019 21:47:10 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS78.soft.gz: 
08-Apr-2019 21:47:10 DEBUG GEOparse - DATABASE: Geo
08-Apr-2019 21:47:10 DEBUG GEOparse - DATASET: GDS78
08-Apr-2019 21:47:10 DEBUG GEOparse - SUBSET: GDS78_1
08-Apr-2019 21:47:10 DEBUG GEOparse - SUBSET: GDS78_2
08-Apr-2019 21:47:10 DEBUG GEOparse - SUBSET: GDS78_3
08-Apr-2019 21:47:10 DEBUG GEOparse - SUBSET: GDS78_4
08-Apr-2019 21:47:10 DEBUG GEOparse - SUBSET: GDS78_5
08-Apr-2019 21:47:10 DEBUG GEOparse - SUBSET: GDS78_6
08-Apr-2019 21:47:10 DEBUG GEOparse - SUBSET: GDS78_7
08-Apr-2019 21:47:10 DEBUG GEOparse - DATASET: GDS78


((24192, 13),
 Index(['ID_REF', 'IDENTIFIER', 'GSM1798', 'GSM1794', 'GSM1801', 'GSM1796',
        'GSM1795', 'GSM1799', 'GSM1792', 'GSM1797', 'GSM1791', 'GSM1793',
        'GSM1800'],
       dtype='object'))

## Unfortunately, there is nothing we can do anymore for merging
## So we now start for analysis from now on

In [54]:
gds_dims = {}
for gds_file in gds_files:
    print(gds_file)
    gse = GEOparse.get_GEO(filepath=gds_file)
    gseData = gse.table
    gds_dims[gds_file.split('/')[-1]] =  gseData.shape

gds_dims

09-Apr-2019 17:33:49 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS1012.soft.gz: 


D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS1012.soft.gz


09-Apr-2019 17:33:49 DEBUG GEOparse - DATABASE: Geo
09-Apr-2019 17:33:49 DEBUG GEOparse - DATASET: GDS1012
09-Apr-2019 17:33:49 DEBUG GEOparse - SUBSET: GDS1012_1
09-Apr-2019 17:33:49 DEBUG GEOparse - SUBSET: GDS1012_2
09-Apr-2019 17:33:49 DEBUG GEOparse - SUBSET: GDS1012_3
09-Apr-2019 17:33:49 DEBUG GEOparse - SUBSET: GDS1012_4
09-Apr-2019 17:33:49 DEBUG GEOparse - SUBSET: GDS1012_5
09-Apr-2019 17:33:49 DEBUG GEOparse - DATASET: GDS1012
09-Apr-2019 17:33:51 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS1020.soft.gz: 
09-Apr-2019 17:33:51 DEBUG GEOparse - DATABASE: Geo
09-Apr-2019 17:33:51 DEBUG GEOparse - DATASET: GDS1020
09-Apr-2019 17:33:51 DEBUG GEOparse - SUBSET: GDS1020_1
09-Apr-2019 17:33:51 DEBUG GEOparse - SUBSET: GDS1020_2
09-Apr-2019 17:33:51 DEBUG GEOparse - SUBSET: GDS1020_3
09-Apr-2019 17:33:51 DEBUG GEOparse - SUBSET: GDS1020_4
09-Apr-2019 17:33:51 DEBUG GEOparse - SUBSET: GDS1020_5
09-Apr-201

D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS1020.soft.gz


09-Apr-2019 17:33:51 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS1022.soft.gz: 
09-Apr-2019 17:33:51 DEBUG GEOparse - DATABASE: Geo
09-Apr-2019 17:33:51 DEBUG GEOparse - DATASET: GDS1022
09-Apr-2019 17:33:51 DEBUG GEOparse - SUBSET: GDS1022_1
09-Apr-2019 17:33:51 DEBUG GEOparse - SUBSET: GDS1022_2
09-Apr-2019 17:33:51 DEBUG GEOparse - SUBSET: GDS1022_3
09-Apr-2019 17:33:51 DEBUG GEOparse - SUBSET: GDS1022_4
09-Apr-2019 17:33:51 DEBUG GEOparse - SUBSET: GDS1022_5
09-Apr-2019 17:33:51 DEBUG GEOparse - SUBSET: GDS1022_6
09-Apr-2019 17:33:51 DEBUG GEOparse - SUBSET: GDS1022_7
09-Apr-2019 17:33:51 DEBUG GEOparse - SUBSET: GDS1022_8
09-Apr-2019 17:33:51 DEBUG GEOparse - SUBSET: GDS1022_9
09-Apr-2019 17:33:51 DEBUG GEOparse - DATASET: GDS1022
09-Apr-2019 17:33:51 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS1023.soft.gz: 
09-Apr-2019 

...

09-Apr-2019 17:50:41 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS989.soft.gz: 
09-Apr-2019 17:50:41 DEBUG GEOparse - DATABASE: Geo
09-Apr-2019 17:50:41 DEBUG GEOparse - DATASET: GDS989
09-Apr-2019 17:50:41 DEBUG GEOparse - SUBSET: GDS989_1
09-Apr-2019 17:50:41 DEBUG GEOparse - SUBSET: GDS989_2
09-Apr-2019 17:50:41 DEBUG GEOparse - SUBSET: GDS989_3
09-Apr-2019 17:50:41 DEBUG GEOparse - SUBSET: GDS989_4
09-Apr-2019 17:50:41 DEBUG GEOparse - SUBSET: GDS989_5
09-Apr-2019 17:50:41 DEBUG GEOparse - SUBSET: GDS989_6
09-Apr-2019 17:50:41 DEBUG GEOparse - SUBSET: GDS989_7
09-Apr-2019 17:50:41 DEBUG GEOparse - SUBSET: GDS989_8
09-Apr-2019 17:50:41 DEBUG GEOparse - SUBSET: GDS989_9
09-Apr-2019 17:50:41 DEBUG GEOparse - SUBSET: GDS989_10
09-Apr-2019 17:50:41 DEBUG GEOparse - DATASET: GDS989
09-Apr-2019 17:50:41 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dat

D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS989.soft.gz
D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS992.soft.gz


09-Apr-2019 17:50:41 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS999.soft.gz: 
09-Apr-2019 17:50:41 DEBUG GEOparse - DATABASE: Geo
09-Apr-2019 17:50:41 DEBUG GEOparse - DATASET: GDS999
09-Apr-2019 17:50:41 DEBUG GEOparse - SUBSET: GDS999_1
09-Apr-2019 17:50:41 DEBUG GEOparse - SUBSET: GDS999_2
09-Apr-2019 17:50:41 DEBUG GEOparse - DATASET: GDS999


D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS999.soft.gz


{'D:\\My BUET Documents\\October 18\\BioInformatics Algorithms\\project\\Datasets\\FINAL_dataset\\gds-soft\\GDS1012.soft.gz': (12625,
  20),
 'D:\\My BUET Documents\\October 18\\BioInformatics Algorithms\\project\\Datasets\\FINAL_dataset\\gds-soft\\GDS1020.soft.gz': (12626,
  35),
 'D:\\My BUET Documents\\October 18\\BioInformatics Algorithms\\project\\Datasets\\FINAL_dataset\\gds-soft\\GDS1022.soft.gz': (12625,
  18),
 'D:\\My BUET Documents\\October 18\\BioInformatics Algorithms\\project\\Datasets\\FINAL_dataset\\gds-soft\\GDS1023.soft.gz': (22283,
  25),
 'D:\\My BUET Documents\\October 18\\BioInformatics Algorithms\\project\\Datasets\\FINAL_dataset\\gds-soft\\GDS1028.soft.gz': (8793,
  16),
 'D:\\My BUET Documents\\October 18\\BioInformatics Algorithms\\project\\Datasets\\FINAL_dataset\\gds-soft\\GDS1036.soft.gz': (22283,
  26),
 'D:\\My BUET Documents\\October 18\\BioInformatics Algorithms\\project\\Datasets\\FINAL_dataset\\gds-soft\\GDS1048.soft.gz': (23916,
  169),
 'D:\\My BUET

In [2]:
# THIS FOLLOWING LIST CONTAINS OUR 711 GENES
genes711 = []
with open('712_known_cancerGenes.txt') as f:
    for line in f:
        line = line.split('\n')[0]
        genes711.append(line)
len(genes711)

711

In [146]:
# THIS CELL FIND NUMBER OF COMMON GENES IN A GDS FILE TO OUR 711 GENES
poop_gds = {}

poop = ["D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS78.soft.gz", "D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS5045.soft.gz"]
for j in poop:
    gse = GEOparse.get_GEO(filepath=j)
    GDS = gse.table
    iden = GDS.iloc[1:,1]
    print (GDS.shape, len(iden), len(genes711))
    
    iden = set(iden)
    geneset = set(genes711)
    
    count = 0
    #common = []
#     for i in iden:
#         if i in genes711 and i not in common:
#             #genes711[idx][1] = 1
#             common.append(i)
#             count += 1
    intersection = list(iden.intersection(geneset))
    count = len(intersection)
    poop_gds[i] = (count, GDS.shape)
    print (count)

# poop = "D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS5045.soft.gz"
# gse5045 = GEOparse.get_GEO(filepath=poop)
# GDS2 = gse5045.table
# iden2 = GDS2.iloc[1:,1]
# print (GDS2.shape, len(iden2), len(genes711))

# count = 0
# common = []
# for i in iden2:
#     if i in genes711 and i not in common:
#         common.append(i)
#         count += 1
#         poop_gds[poop] = (count, GDS2.shape)
# print (count, common[:10])


lol = Counter(poop_gds)
print (lol.most_common(1))

10-Apr-2019 02:18:14 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS78.soft.gz: 
10-Apr-2019 02:18:14 DEBUG GEOparse - DATABASE: Geo
10-Apr-2019 02:18:14 DEBUG GEOparse - DATASET: GDS78
10-Apr-2019 02:18:14 DEBUG GEOparse - SUBSET: GDS78_1
10-Apr-2019 02:18:14 DEBUG GEOparse - SUBSET: GDS78_2
10-Apr-2019 02:18:14 DEBUG GEOparse - SUBSET: GDS78_3
10-Apr-2019 02:18:14 DEBUG GEOparse - SUBSET: GDS78_4
10-Apr-2019 02:18:14 DEBUG GEOparse - SUBSET: GDS78_5
10-Apr-2019 02:18:14 DEBUG GEOparse - SUBSET: GDS78_6
10-Apr-2019 02:18:14 DEBUG GEOparse - SUBSET: GDS78_7
10-Apr-2019 02:18:14 DEBUG GEOparse - DATASET: GDS78
10-Apr-2019 02:18:14 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS5045.soft.gz: 
10-Apr-2019 02:18:14 DEBUG GEOparse - DATABASE: Geo
10-Apr-2019 02:18:14 DEBUG GEOparse - DATASET: GDS5045
10-Apr-2019 02:18:14 DEBUG GEOparse -

(24192, 13) 24191 711
325
(54675, 14) 54674 711
687
[(4, (687, (54675, 14)))]


In [6]:
#THIS IS THE MAIN SCRIPT TO FIND OUT HOW MANY GENES FROM 711_GENES ARE COMMON IN EACH GDS FILE

gds_711_count = {}
gene_set = set(genes711)

for gds_file in gds_files:
    #print(gds_file)
    count = 0
    #temp_common_list = []
    gds_obj = GEOparse.get_GEO(filepath=gds_file)
    gseDataframe = gds_obj.table
    
    gseDataIdentifiers = gseDataframe.iloc[1:,1]
    gseDataIdentifiers = set(gseDataIdentifiers)

    result = gseDataIdentifiers.intersection(gene_set)
    count = len(list(result))

    gds_711_count[gds_file] = (count, gseDataframe.shape)

#This following max_711genes_gds contains the gds files in decresing order of how many 711genes are inside each of them
max_711genes_gds = Counter(gds_711_count)
max_711genes_gds.most_common()

10-Apr-2019 04:28:06 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS1012.soft.gz: 
10-Apr-2019 04:28:06 DEBUG GEOparse - DATABASE: Geo
10-Apr-2019 04:28:06 DEBUG GEOparse - DATASET: GDS1012
10-Apr-2019 04:28:06 DEBUG GEOparse - SUBSET: GDS1012_1
10-Apr-2019 04:28:06 DEBUG GEOparse - SUBSET: GDS1012_2
10-Apr-2019 04:28:06 DEBUG GEOparse - SUBSET: GDS1012_3
10-Apr-2019 04:28:06 DEBUG GEOparse - SUBSET: GDS1012_4
10-Apr-2019 04:28:06 DEBUG GEOparse - SUBSET: GDS1012_5
10-Apr-2019 04:28:06 DEBUG GEOparse - DATASET: GDS1012
10-Apr-2019 04:28:06 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS1020.soft.gz: 
10-Apr-2019 04:28:06 DEBUG GEOparse - DATABASE: Geo
10-Apr-2019 04:28:06 DEBUG GEOparse - DATASET: GDS1020
10-Apr-2019 04:28:06 DEBUG GEOparse - SUBSET: GDS1020_1
10-Apr-2019 04:28:06 DEBUG GEOparse - SUBSET: GDS1020_2
10-Apr-2019 04:28

10-Apr-2019 04:28:12 DEBUG GEOparse - SUBSET: GDS1096_34
10-Apr-2019 04:28:12 DEBUG GEOparse - SUBSET: GDS1096_35
10-Apr-2019 04:28:12 DEBUG GEOparse - SUBSET: GDS1096_36
10-Apr-2019 04:28:12 DEBUG GEOparse - DATASET: GDS1096
10-Apr-2019 04:28:12 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS1112.soft.gz: 
10-Apr-2019 04:28:12 DEBUG GEOparse - DATABASE: Geo
10-Apr-2019 04:28:12 DEBUG GEOparse - DATASET: GDS1112
10-Apr-2019 04:28:12 DEBUG GEOparse - SUBSET: GDS1112_1
10-Apr-2019 04:28:12 DEBUG GEOparse - SUBSET: GDS1112_2
10-Apr-2019 04:28:12 DEBUG GEOparse - DATASET: GDS1112
10-Apr-2019 04:28:13 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS12.soft.gz: 
10-Apr-2019 04:28:13 DEBUG GEOparse - DATABASE: Geo
10-Apr-2019 04:28:13 DEBUG GEOparse - DATASET: GDS12
10-Apr-2019 04:28:13 DEBUG GEOparse - SUBSET: GDS12_1
10-Apr-2019 04:28:13 

10-Apr-2019 04:28:15 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS1249.soft.gz: 
10-Apr-2019 04:28:15 DEBUG GEOparse - DATABASE: Geo
10-Apr-2019 04:28:15 DEBUG GEOparse - DATASET: GDS1249
10-Apr-2019 04:28:15 DEBUG GEOparse - SUBSET: GDS1249_1
10-Apr-2019 04:28:15 DEBUG GEOparse - SUBSET: GDS1249_2
10-Apr-2019 04:28:15 DEBUG GEOparse - SUBSET: GDS1249_3
10-Apr-2019 04:28:15 DEBUG GEOparse - SUBSET: GDS1249_4
10-Apr-2019 04:28:15 DEBUG GEOparse - SUBSET: GDS1249_5
10-Apr-2019 04:28:15 DEBUG GEOparse - SUBSET: GDS1249_6
10-Apr-2019 04:28:15 DEBUG GEOparse - SUBSET: GDS1249_7
10-Apr-2019 04:28:15 DEBUG GEOparse - DATASET: GDS1249
10-Apr-2019 04:28:16 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS1250.soft.gz: 
10-Apr-2019 04:28:16 DEBUG GEOparse - DATABASE: Geo
10-Apr-2019 04:28:16 DEBUG GEOparse - DATASET: GDS1250
10-Apr-2019 04:28

10-Apr-2019 04:28:20 DEBUG GEOparse - SUBSET: GDS1289_2
10-Apr-2019 04:28:20 DEBUG GEOparse - SUBSET: GDS1289_3
10-Apr-2019 04:28:20 DEBUG GEOparse - SUBSET: GDS1289_4
10-Apr-2019 04:28:20 DEBUG GEOparse - SUBSET: GDS1289_5
10-Apr-2019 04:28:20 DEBUG GEOparse - SUBSET: GDS1289_6
10-Apr-2019 04:28:20 DEBUG GEOparse - SUBSET: GDS1289_7
10-Apr-2019 04:28:20 DEBUG GEOparse - DATASET: GDS1289
10-Apr-2019 04:28:20 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS1290.soft.gz: 
10-Apr-2019 04:28:20 DEBUG GEOparse - DATABASE: Geo
10-Apr-2019 04:28:20 DEBUG GEOparse - DATASET: GDS1290
10-Apr-2019 04:28:20 DEBUG GEOparse - SUBSET: GDS1290_1
10-Apr-2019 04:28:20 DEBUG GEOparse - SUBSET: GDS1290_2
10-Apr-2019 04:28:20 DEBUG GEOparse - SUBSET: GDS1290_3
10-Apr-2019 04:28:20 DEBUG GEOparse - SUBSET: GDS1290_4
10-Apr-2019 04:28:20 DEBUG GEOparse - SUBSET: GDS1290_5
10-Apr-2019 04:28:20 DEBUG GEOparse - SUBSET: GDS1290_6
10-Ap

10-Apr-2019 04:28:22 DEBUG GEOparse - SUBSET: GDS1325_6
10-Apr-2019 04:28:22 DEBUG GEOparse - SUBSET: GDS1325_7
10-Apr-2019 04:28:22 DEBUG GEOparse - DATASET: GDS1325
10-Apr-2019 04:28:22 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS1326.soft.gz: 
10-Apr-2019 04:28:22 DEBUG GEOparse - DATABASE: Geo
10-Apr-2019 04:28:22 DEBUG GEOparse - DATASET: GDS1326
10-Apr-2019 04:28:22 DEBUG GEOparse - SUBSET: GDS1326_1
10-Apr-2019 04:28:22 DEBUG GEOparse - SUBSET: GDS1326_2
10-Apr-2019 04:28:22 DEBUG GEOparse - SUBSET: GDS1326_3
10-Apr-2019 04:28:22 DEBUG GEOparse - SUBSET: GDS1326_4
10-Apr-2019 04:28:22 DEBUG GEOparse - DATASET: GDS1326
10-Apr-2019 04:28:22 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS1327.soft.gz: 
10-Apr-2019 04:28:22 DEBUG GEOparse - DATABASE: Geo
10-Apr-2019 04:28:22 DEBUG GEOparse - DATASET: GDS1327
10-Apr-2019 04:28:

...

10-Apr-2019 04:42:32 DEBUG GEOparse - DATABASE: Geo
10-Apr-2019 04:42:32 DEBUG GEOparse - DATASET: GDS846
10-Apr-2019 04:42:32 DEBUG GEOparse - SUBSET: GDS846_1
10-Apr-2019 04:42:32 DEBUG GEOparse - SUBSET: GDS846_2
10-Apr-2019 04:42:32 DEBUG GEOparse - SUBSET: GDS846_3
10-Apr-2019 04:42:32 DEBUG GEOparse - SUBSET: GDS846_4
10-Apr-2019 04:42:32 DEBUG GEOparse - SUBSET: GDS846_5
10-Apr-2019 04:42:32 DEBUG GEOparse - SUBSET: GDS846_6
10-Apr-2019 04:42:32 DEBUG GEOparse - DATASET: GDS846
10-Apr-2019 04:42:33 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS847.soft.gz: 
10-Apr-2019 04:42:33 DEBUG GEOparse - DATABASE: Geo
10-Apr-2019 04:42:33 DEBUG GEOparse - DATASET: GDS847
10-Apr-2019 04:42:33 DEBUG GEOparse - SUBSET: GDS847_1
10-Apr-2019 04:42:33 DEBUG GEOparse - SUBSET: GDS847_2
10-Apr-2019 04:42:33 DEBUG GEOparse - SUBSET: GDS847_3
10-Apr-2019 04:42:33 DEBUG GEOparse - SUBSET: GDS847_4
10-Apr-2019 04:42:33 DEB

10-Apr-2019 04:42:34 DEBUG GEOparse - DATASET: GDS884
10-Apr-2019 04:42:34 DEBUG GEOparse - SUBSET: GDS884_1
10-Apr-2019 04:42:34 DEBUG GEOparse - SUBSET: GDS884_2
10-Apr-2019 04:42:34 DEBUG GEOparse - SUBSET: GDS884_3
10-Apr-2019 04:42:34 DEBUG GEOparse - SUBSET: GDS884_4
10-Apr-2019 04:42:34 DEBUG GEOparse - SUBSET: GDS884_5
10-Apr-2019 04:42:34 DEBUG GEOparse - SUBSET: GDS884_6
10-Apr-2019 04:42:34 DEBUG GEOparse - SUBSET: GDS884_7
10-Apr-2019 04:42:34 DEBUG GEOparse - SUBSET: GDS884_8
10-Apr-2019 04:42:34 DEBUG GEOparse - SUBSET: GDS884_9
10-Apr-2019 04:42:34 DEBUG GEOparse - DATASET: GDS884
10-Apr-2019 04:42:35 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS885.soft.gz: 
10-Apr-2019 04:42:35 DEBUG GEOparse - DATABASE: Geo
10-Apr-2019 04:42:35 DEBUG GEOparse - DATASET: GDS885
10-Apr-2019 04:42:35 DEBUG GEOparse - SUBSET: GDS885_1
10-Apr-2019 04:42:35 DEBUG GEOparse - SUBSET: GDS885_2
10-Apr-2019 04:42:35 

10-Apr-2019 04:42:37 DEBUG GEOparse - DATABASE: Geo
10-Apr-2019 04:42:37 DEBUG GEOparse - DATASET: GDS918
10-Apr-2019 04:42:37 DEBUG GEOparse - SUBSET: GDS918_1
10-Apr-2019 04:42:37 DEBUG GEOparse - SUBSET: GDS918_2
10-Apr-2019 04:42:37 DEBUG GEOparse - SUBSET: GDS918_3
10-Apr-2019 04:42:37 DEBUG GEOparse - SUBSET: GDS918_4
10-Apr-2019 04:42:37 DEBUG GEOparse - SUBSET: GDS918_5
10-Apr-2019 04:42:37 DEBUG GEOparse - SUBSET: GDS918_6
10-Apr-2019 04:42:37 DEBUG GEOparse - DATASET: GDS918
10-Apr-2019 04:42:37 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS940.soft.gz: 
10-Apr-2019 04:42:37 DEBUG GEOparse - DATABASE: Geo
10-Apr-2019 04:42:37 DEBUG GEOparse - DATASET: GDS940
10-Apr-2019 04:42:37 DEBUG GEOparse - SUBSET: GDS940_1
10-Apr-2019 04:42:37 DEBUG GEOparse - SUBSET: GDS940_2
10-Apr-2019 04:42:37 DEBUG GEOparse - DATASET: GDS940
10-Apr-2019 04:42:37 INFO GEOparse - Parsing D:\My BUET Documents\October 18\Bio

10-Apr-2019 04:42:39 DEBUG GEOparse - SUBSET: GDS968_53
10-Apr-2019 04:42:39 DEBUG GEOparse - SUBSET: GDS968_54
10-Apr-2019 04:42:39 DEBUG GEOparse - SUBSET: GDS968_55
10-Apr-2019 04:42:39 DEBUG GEOparse - SUBSET: GDS968_56
10-Apr-2019 04:42:39 DEBUG GEOparse - SUBSET: GDS968_57
10-Apr-2019 04:42:39 DEBUG GEOparse - SUBSET: GDS968_58
10-Apr-2019 04:42:39 DEBUG GEOparse - SUBSET: GDS968_59
10-Apr-2019 04:42:39 DEBUG GEOparse - SUBSET: GDS968_60
10-Apr-2019 04:42:39 DEBUG GEOparse - SUBSET: GDS968_61
10-Apr-2019 04:42:39 DEBUG GEOparse - SUBSET: GDS968_62
10-Apr-2019 04:42:39 DEBUG GEOparse - SUBSET: GDS968_63
10-Apr-2019 04:42:39 DEBUG GEOparse - SUBSET: GDS968_64
10-Apr-2019 04:42:39 DEBUG GEOparse - DATASET: GDS968
10-Apr-2019 04:42:40 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS969.soft.gz: 
10-Apr-2019 04:42:40 DEBUG GEOparse - DATABASE: Geo
10-Apr-2019 04:42:40 DEBUG GEOparse - DATASET: GDS969
10-Apr-2

[('D:\\My BUET Documents\\October 18\\BioInformatics Algorithms\\project\\Datasets\\FINAL_dataset\\gds-soft\\GDS5037.soft.gz',
  (698, (41108, 110))),
 ('D:\\My BUET Documents\\October 18\\BioInformatics Algorithms\\project\\Datasets\\FINAL_dataset\\gds-soft\\GDS5176.soft.gz',
  (698, (41108, 59))),
 ('D:\\My BUET Documents\\October 18\\BioInformatics Algorithms\\project\\Datasets\\FINAL_dataset\\gds-soft\\GDS3897.soft.gz',
  (698, (41108, 46))),
 ('D:\\My BUET Documents\\October 18\\BioInformatics Algorithms\\project\\Datasets\\FINAL_dataset\\gds-soft\\GDS5206.soft.gz',
  (698, (41108, 20))),
 ('D:\\My BUET Documents\\October 18\\BioInformatics Algorithms\\project\\Datasets\\FINAL_dataset\\gds-soft\\GDS6010.soft.gz',
  (698, (41108, 20))),
 ('D:\\My BUET Documents\\October 18\\BioInformatics Algorithms\\project\\Datasets\\FINAL_dataset\\gds-soft\\GDS4807.soft.gz',
  (698, (41108, 18))),
 ('D:\\My BUET Documents\\October 18\\BioInformatics Algorithms\\project\\Datasets\\FINAL_dataset\\

In [7]:
max_711genes_gds.most_common(5)

[('D:\\My BUET Documents\\October 18\\BioInformatics Algorithms\\project\\Datasets\\FINAL_dataset\\gds-soft\\GDS5037.soft.gz',
  (698, (41108, 110))),
 ('D:\\My BUET Documents\\October 18\\BioInformatics Algorithms\\project\\Datasets\\FINAL_dataset\\gds-soft\\GDS5176.soft.gz',
  (698, (41108, 59))),
 ('D:\\My BUET Documents\\October 18\\BioInformatics Algorithms\\project\\Datasets\\FINAL_dataset\\gds-soft\\GDS3897.soft.gz',
  (698, (41108, 46))),
 ('D:\\My BUET Documents\\October 18\\BioInformatics Algorithms\\project\\Datasets\\FINAL_dataset\\gds-soft\\GDS5206.soft.gz',
  (698, (41108, 20))),
 ('D:\\My BUET Documents\\October 18\\BioInformatics Algorithms\\project\\Datasets\\FINAL_dataset\\gds-soft\\GDS6010.soft.gz',
  (698, (41108, 20)))]

In [8]:
# NOW LETS TRY TO MERGE THE FIRST 5 OF THESE GDS FILES
new_gds_with_common711 = max_711genes_gds.most_common(5)

gds_obj = GEOparse.get_GEO(filepath=new_gds_with_common711[0][0])
new_merged_Dataset = gds_obj.table

for file in new_gds_with_common711[1:3]:
    gds_obj = GEOparse.get_GEO(filepath=file[0])
    gseDataframe = gds_obj.table
    
    print ("previous shape: %d,%d"%(new_merged_Dataset.shape[0], new_merged_Dataset.shape[1]))
    new_merged_Dataset = pd.merge(new_merged_Dataset, gseDataframe, on='IDENTIFIER', how='outer')
    print ("new shape: %d,%d\n\n"%(new_merged_Dataset.shape[0], new_merged_Dataset.shape[1]))

10-Apr-2019 04:42:42 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS5037.soft.gz: 
10-Apr-2019 04:42:42 DEBUG GEOparse - DATABASE: Geo
10-Apr-2019 04:42:42 DEBUG GEOparse - DATASET: GDS5037
10-Apr-2019 04:42:42 DEBUG GEOparse - SUBSET: GDS5037_1
10-Apr-2019 04:42:42 DEBUG GEOparse - SUBSET: GDS5037_2
10-Apr-2019 04:42:42 DEBUG GEOparse - SUBSET: GDS5037_3
10-Apr-2019 04:42:42 DEBUG GEOparse - SUBSET: GDS5037_4
10-Apr-2019 04:42:42 DEBUG GEOparse - SUBSET: GDS5037_5
10-Apr-2019 04:42:42 DEBUG GEOparse - DATASET: GDS5037
10-Apr-2019 04:42:45 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS5176.soft.gz: 
10-Apr-2019 04:42:45 DEBUG GEOparse - DATABASE: Geo
10-Apr-2019 04:42:45 DEBUG GEOparse - DATASET: GDS5176
10-Apr-2019 04:42:45 DEBUG GEOparse - SUBSET: GDS5176_1
10-Apr-2019 04:42:45 DEBUG GEOparse - SUBSET: GDS5176_2
10-Apr-2019 04:42

previous shape: 41108,110


10-Apr-2019 04:42:46 INFO GEOparse - Parsing D:\My BUET Documents\October 18\BioInformatics Algorithms\project\Datasets\FINAL_dataset\gds-soft\GDS3897.soft.gz: 
10-Apr-2019 04:42:46 DEBUG GEOparse - DATABASE: Geo
10-Apr-2019 04:42:46 DEBUG GEOparse - DATASET: GDS3897
10-Apr-2019 04:42:46 DEBUG GEOparse - SUBSET: GDS3897_1
10-Apr-2019 04:42:46 DEBUG GEOparse - SUBSET: GDS3897_2
10-Apr-2019 04:42:46 DEBUG GEOparse - SUBSET: GDS3897_3
10-Apr-2019 04:42:46 DEBUG GEOparse - SUBSET: GDS3897_4
10-Apr-2019 04:42:46 DEBUG GEOparse - SUBSET: GDS3897_5
10-Apr-2019 04:42:46 DEBUG GEOparse - SUBSET: GDS3897_6
10-Apr-2019 04:42:46 DEBUG GEOparse - SUBSET: GDS3897_7
10-Apr-2019 04:42:46 DEBUG GEOparse - SUBSET: GDS3897_8
10-Apr-2019 04:42:46 DEBUG GEOparse - SUBSET: GDS3897_9
10-Apr-2019 04:42:46 DEBUG GEOparse - SUBSET: GDS3897_10
10-Apr-2019 04:42:46 DEBUG GEOparse - SUBSET: GDS3897_11
10-Apr-2019 04:42:46 DEBUG GEOparse - SUBSET: GDS3897_12
10-Apr-2019 04:42:46 DEBUG GEOparse - SUBSET: GDS3897_13


new shape: 73194,168


previous shape: 73194,168
new shape: 182222,213




## We have got a little merged dataset combinig of 4 GDS files. 
## We'll try to run our models in this dataset now, in the next file